In [1]:
#!pip install gmaps
#!pip install networkx
#!pip install geopy
!pip install gmplot
import networkx as nx
with open('apiKey.txt') as f:
    api_key = f.readline()
    f.close

     |████████████████████████████████| 122kB 3.3MB/s eta 0:00:01
  Created wheel for gmplot: filename=gmplot-1.2.0-cp37-none-any.whl size=143766 sha256=8feda0ea8a66c34d059500ff50a4ca9636a96df3c8d8c741c9066021a1bee68f
  Stored in directory: /home/jovyan/.cache/pip/wheels/81/6a/76/4dd6a7cc310ba765894159ee84871e8cd55221d82ef14b81a1
Successfully built gmplot
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import gmaps
gmaps.configure(api_key=api_key)

In [3]:
skytree_coordinates = (35.710064, 139.810701)
gmaps.figure(center=skytree_coordinates, zoom_level=18)

Figure(layout=FigureLayout(height='420px'))

In [4]:
import gmaps.datasets
# Get the dataset
earthquake_df = gmaps.datasets.load_dataset_as_df('earthquakes')
#Get the locations from the data set
locations = earthquake_df[['latitude', 'longitude']]
#Get the magnitude from the data
weights = earthquake_df['magnitude']
#Set up your map
fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights))
fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
#Define location 1 and 2
Durango = (37.2753,-107.880067)
SF = (37.7749,-122.419416)
#Create the map
fig = gmaps.figure()
#create the layer
layer = gmaps.directions.Directions(Durango, SF,mode='airplane')
#Add the layer
fig.add_layer(layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
earthquake_df

,latitude,longitude,magnitude
0,65.193300,-149.072500,1.70
1,38.791832,-122.780830,2.10
2,38.818001,-122.792168,0.48
3,33.601667,-116.727667,0.78
4,37.378334,-118.520836,3.64
...,...,...,...
8599,19.331500,-155.182200,0.80
8600,41.819700,-119.622400,2.20
8601,38.771999,-122.720497,1.91
8602,34.608333,-118.641333,1.93


In [7]:
# Map centered on taipei 101
fig = gmaps.figure(center=(25.033967, 121.564473), zoom_level=12)
fig.add_layer(gmaps.transit_layer())
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
# Map centered on London
fig = gmaps.figure(center=(25.033967, 121.564473), zoom_level=12)
fig.add_layer(gmaps.traffic_layer())
fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
import ipywidgets as widgets
import geopy
import gmaps
gmaps.configure(api_key=api_key)
class ReverseGeocoder(object):
    """
    Jupyter widget for finding addresses.
    The user places markers on a map. For each marker,
    we use `geopy` to find the nearest address to that
    marker, and write that address in a text box.
    """
    def __init__(self):
        self._figure = gmaps.figure()
        self._drawing = gmaps.drawing_layer()
        self._drawing.on_new_feature(self._new_feature_callback)
        self._figure.add_layer(self._drawing)
        self._address_box = widgets.Text(
        description='Address: ',
        disabled=True,
        layout={'width': '95%', 'margin': '10px 0 0 0'}
        )
        self._geocoder = geopy.geocoders.GoogleV3(api_key=api_key)
        self._container = widgets.VBox([self._figure, self._address_box])
    def _get_location_details(self, location):
        return self._geocoder.reverse(location, exactly_one=True)
    def _clear_address_box(self):
        self._address_box.value = ''
    def _show_address(self, location):
        location_details = self._get_location_details(location)
        if location_details is None:
            self._address_box.value = 'No address found'
        else:
            self._address_box.value = location_details.address
    def _new_feature_callback(self, feature):
        try:
            location = feature.location
        except AttributeError:
            return # Not a marker
        # Clear address box to signify to the user that something is happening
        self._clear_address_box()
        # Remove all markers other than the one that has just been added.
        self._drawing.features = [feature]
        # Compute the address and display it
        self._show_address(location)
    def render(self):
        return self._container
ReverseGeocoder().render()

In [10]:
import gmaps.datasets
df = gmaps.datasets.load_dataset_as_df('acled_africa_by_year')
df.head()

,year,latitude,longitude,fatalities
0,2018,32.0837,48.4100,0
1,2018,30.6492,48.6650,0
2,2018,35.7757,51.4721,0
3,2018,14.6008,-0.7190,1
4,2018,14.2100,-1.8315,1


In [11]:
from IPython.display import display
import ipywidgets as widgets
import gmaps
gmaps.configure(api_key=api_key)
class AcledExplorer(object):
    """
    Jupyter widget for exploring the ACLED dataset.
    The user uses the slider to choose a year. This renders
    a heatmap of civilian victims in that year.
    """
    def __init__(self, df):
        self._df = df
        self._heatmap = None
        self._slider = None
        initial_year = min(self._df['year'])
        title_widget = widgets.HTML(
            '<h3>Civilian casualties in Africa, by year</h3>'
            '<h4>Data from <a href="https://www.acleddata.com/">ACLED project</a></h4>'
        )
        map_figure = self._render_map(initial_year)
        controls = self._render_controls(initial_year)
        self._container = widgets.VBox([title_widget, controls, map_figure])
    def render(self):
        display(self._container)
    def _on_year_change(self, change):
        year = self._slider.value
        self._heatmap.locations = self._locations_for_year(year)
        self._total_box.value = self._total_casualties_text_for_year(year)
        return self._container
    def _render_map(self, initial_year):
        fig = gmaps.figure(map_type='HYBRID')
        self._heatmap = gmaps.heatmap_layer(
        self._locations_for_year(initial_year),
        max_intensity=100,
        point_radius=8
        )
        fig.add_layer(self._heatmap)
        return fig
    def _render_controls(self, initial_year):
        self._slider = widgets.IntSlider(
        value=initial_year,
        min=min(self._df['year']),
        max=max(self._df['year']),
        description='Year',
        continuous_update=False
        )
        self._total_box = widgets.Label(
        value=self._total_casualties_text_for_year(initial_year)
        )
        self._slider.observe(self._on_year_change, names='value')
        controls = widgets.HBox(
        [self._slider, self._total_box],
        layout={'justify_content': 'space-between'}
        )
        return controls
    def _locations_for_year(self, year):
        return self._df[self._df['year'] == year][['latitude', 'longitude']]
    def _total_casualties_for_year(self, year):
        return int(self._df[self._df['year'] == year]['year'].count())
    def _total_casualties_text_for_year(self, year):
        return '{} civilian casualties'.format(self._total_casualties_for_year(year))
AcledExplorer(df).render()

In [12]:
from IPython.display import display
import ipywidgets as widgets
import gmaps
import gmaps.datasets


gmaps.configure(api_key=api_key)

class OutletExplorer(object):
    def __init__(self, df):
        """
        Jupyter widget for exploring KFC and Starbucks outlets
        Using checkboxes, the user chooses whether to include
        Starbucks, KFC outlets, both or neither.
        """
        self._df = df
        self._symbol_layer = None
        
        self._starbucks_symbols = self._create_symbols_for_chain(
            'starbucks', 'rgba(0, 150, 0, 0.4)')
        self._kfc_symbols = self._create_symbols_for_chain(
            'kfc', 'rgba(150, 0, 0, 0.4)')
        
        title_widget = widgets.HTML(
            '<h3>Explore KFC and Starbucks locations</h3>'
            '<h4>Data from <a href="http://ratings.food.gov.uk">UK Food Standards Agency</a></h4>'
        )
        controls = self._render_controls(True, True)
        map_figure = self._render_map(True, True)
        self._container = widgets.VBox(
            [title_widget, controls, map_figure])
    def render(self):
        """ Render the widget """
        display(self._container)
    def _render_map(self, initial_include_starbucks, initial_include_kfc):
        """ Render the initial map """
        fig = gmaps.figure(layout={'height': '500px'})
        symbols = self._generate_symbols(True, True)
        self._symbol_layer = gmaps.Markers(markers=symbols)
        fig.add_layer(self._symbol_layer)
        return fig
    def _render_controls(
        self,
        initial_include_starbucks,
        initial_include_kfc
        ):
        """ Render the checkboxes """
        self._starbucks_checkbox = widgets.Checkbox(
        value=initial_include_starbucks,
        description='Starbucks'
        )
        self._kfc_checkbox = widgets.Checkbox(
        value=initial_include_kfc,
        description='KFC'
        )
        self._starbucks_checkbox.observe(
        self._on_controls_change, names='value')
        self._kfc_checkbox.observe(
        self._on_controls_change, names='value')
        controls = widgets.VBox(
        [self._starbucks_checkbox, self._kfc_checkbox])
        return controls
    def _on_controls_change(self, obj):
        """
        Called when the checkboxes change
        This method builds the list of symbols to include on the map,
        based on the current checkbox values. It then updates the
        symbol layer with the new symbol list.
        """
        include_starbucks = self._starbucks_checkbox.value
        include_kfc = self._kfc_checkbox.value
        symbols = self._generate_symbols(
        include_starbucks, include_kfc)
        # Update the layer with the new symbols:
        self._symbol_layer.markers = symbols
    def _generate_symbols(self, include_starbucks, include_kfc):
        """ Generate the list of symbols to includs """
        symbols = []
        if include_starbucks:
            symbols.extend(self._starbucks_symbols)
        if include_kfc:
            symbols.extend(self._kfc_symbols)
        return symbols
    def _create_symbols_for_chain(self, chain, color):
        chain_df = self._df[self._df['chain_name'] == chain]
        symbols = [
                gmaps.Symbol(
                location=(latitude, longitude),
                stroke_color=color,
                fill_color=color,
                scale=2
                )
                for latitude, longitude in
                zip(chain_df["latitude"], chain_df["longitude"])
                ]
        return symbols

df = gmaps.datasets.load_dataset_as_df("starbucks_kfc_uk")
OutletExplorer(df).render()

In [13]:
from ipywidgets.embed import embed_minimal_html
import gmaps
gmaps.configure(api_key=api_key)
skytree_coordinates = (35.710064, 139.810701)
fig = gmaps.figure(center=skytree_coordinates, zoom_level=12)
embed_minimal_html('skytree.html', views=[fig])

In [20]:
#!pip install networkx
import networkx as nx
import math

def calDist(x1,x2,y1,y2):
    dist = math.sqrt((x2 - x1)**2 + (y2 - y1)**2)
    return dist

skytree=(35.710064, 139.810701)
gundam=(35.625123, 139.775449)
hills=(35.660241, 139.730076)
temple=(35.676400, 139.699325)
palace=(35.6844651,139.7537082)
'''
G = nx.MultiGraph()
elist = [('skytree','gundam',calDist(skytree[0],skytree[1],gundam[0],gundam[1])),
         ('skytree','hills',calDist(skytree[0],skytree[1],hills[0],hills[1])),
         ('skytree','temple',calDist(skytree[0],skytree[1],temple[0],temple[1])),
         ('skytree','palace',calDist(skytree[0],skytree[1],palace[0],palace[1])),
         ('gundam','hills',calDist(gundam[0],gundam[1],hills[0],hills[1])),
         ('gundam','temple',calDist(gundam[0],gundam[1],temple[0],temple[1])),
         ('gundam','palace',calDist(gundam[0],gundam[1],palace[0],palace[1])),
         ('hills','temple',calDist(hills[0],hills[1],temple[0],temple[1])),
         ('hills','palace',calDist(hills[0],hills[1],palace[0],palace[1])),
         ('temple','palace',calDist(temple[0],temple[1],palace[0],palace[1]))
        ]
G.add_weighted_edges_from(elist)
dict(nx.all_pairs_dijkstra(G))
'''
fig = gmaps.figure()
locations = [skytree,gundam,hills,temple]
marker_layer = gmaps.marker_layer(
    locations,
    hover_text=['天空樹', '台場鋼彈城', '6本木之丘', '明治神宮'],
    label = ['ST','DC','RH','MS']
)
fig.add_layer(marker_layer)
fig
embed_minimal_html('marker.html', views=[fig])

In [15]:
#fig = gmaps.figure()
#start = (35.710064, 139.810701)
#end = start
#waypoints=[(35.625123, 139.775449),(35.660241, 139.730076),(35.676400, 139.699325)]
#directions = gmaps.directions_layer(start, 
#                                    end,
#                                    waypoints=waypoints,
#                                    show_markers=False, 
#                                    show_route=True,
#                                    stroke_opacity=1, 
#                                    stroke_weight=4,)
#fig.add_layer(directions)
#embed_minimal_html('skytreeRoute.html', views=[fig])

In [16]:
#fig = gmaps.figure(center=(35.710064, 139.810701), zoom_level=12)
#fig.add_layer(gmaps.transit_layer())
#fig
#embed_minimal_html('skytreeTransit.html', views=[fig])

In [17]:
# Import gmplot library.
from gmplot import *
# Place map
# First two arugments are the geogrphical coordinates .i.e. Latitude and Longitude
#and the zoom resolution.
gmap=gmplot.GoogleMapPlotter(17.438139, 78.39583, 18)
# Because google maps is not a free service now, you need to get an api key. Else commenting
# below line you will see the maps with "For Development Purpose Only" on the screen and maps
# with low resolution.
#gmap.apikey = "Your_API_KEY"
gmap.apikey = api_key
# Location where you want to save your file.
gmap.draw( "E:\\DB103RichardC\\python\\190826PyETL\\map.html" )

In [18]:
# import gmplot package
import gmplot
#Set different latitude and longitude points
a=(35.710064, 139.810701)
b=(35.625123, 139.775449)
c=(35.660241, 139.730076)
d=(35.676400, 139.699325)
tokyo_lats, tokyo_lons = zip(*[a,b,c,d])

#declare the center of the map, and how much we want the map zoomed in
gmap3 = gmplot.GoogleMapPlotter(35.660241, 139.730076,11)
# Scatter map
gmap3.scatter( tokyo_lats, tokyo_lons, '#FF0000',size = 50, marker = True )
# Plot method Draw a line in between given coordinates
gmap3.plot(tokyo_lats, tokyo_lons, 'cornflowerblue', edge_width = 3.0)
#Your Google_API_Key
gmap3.apikey= api_key
# save it to html
gmap3.draw(r"E:\\DB103RichardC\\python\\190826PyETL\\map.html")

In [19]:
# import folium package 
import folium 

my_map3 = folium.Map(location = [28.5011226, 77.4099794], zoom_start = 15) 

# Pass a string in popup parameter 
folium.Marker([28.5011226, 77.4099794], popup = ' Geeksforgeeks.org ').add_to(my_map3) 


my_map3.save(" my_map3.html ") 

ModuleNotFoundError: No module named 'folium'

In [ ]:
from gmplot import gmplot

# Place map
gmap = gmplot.GoogleMapPlotter(37.766956, -122.438481, 13)

# Polygon
golden_gate_park_lats, golden_gate_park_lons = zip(*[
    (37.771269, -122.511015),
    (37.773495, -122.464830),
    (37.774797, -122.454538),
    (37.771988, -122.454018),
    (37.773646, -122.440979),
    (37.772742, -122.440797),
    (37.771096, -122.453889),
    (37.768669, -122.453518),
    (37.766227, -122.460213),
    (37.764028, -122.510347),
    (37.771269, -122.511015)
    ])
gmap.plot(golden_gate_park_lats, golden_gate_park_lons, 'cornflowerblue', edge_width=10)

# Scatter points
top_attraction_lats, top_attraction_lons = zip(*[
    (37.769901, -122.498331),
    (37.768645, -122.475328),
    (37.771478, -122.468677),
    (37.769867, -122.466102),
    (37.767187, -122.467496),
    (37.770104, -122.470436)
    ])
gmap.scatter(top_attraction_lats, top_attraction_lons, '#3B0B39', size=40, marker=False)

# Marker
hidden_gem_lat, hidden_gem_lon = 37.770776, -122.461689
gmap.marker(hidden_gem_lat, hidden_gem_lon, 'cornflowerblue')
gmap.apikey=api_key
# Draw
gmap.draw("my_map.html")